# RandomForest 隨機森林
- [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)，隨機森林的重點是「**隨機**」   
- 隨機：隨機放棄 10~20% 的訓練資料 
- 森林：這裡指的是 Decision Tree，但也可以是 Naive_Bayes 或 KNN
- **隨機森林** 是 **決策樹** 的升級版

-------------------------------------------------------------------------------------- 
### Ensemble learning（集成學習）
1. **Boosting（連續組合）**：
  - 分類器 classifier：  
    一個訓練完，才訓練下一個；所以每個 classifier 都會依賴上一個 classifier   

    classifier_1 -> classifier_2 -> classifier_3 -> 結果  

  - 訓練樣本 train_df：  
    每次選擇的 train_df 都是同一組，但每次每個訓練樣本的權重不同；  
    **權重**是依據上次的訓練結果，上次分類錯誤的樣本（misclassified data），在下次訓練時權重會提高  

  - 例如：`AdaBoost`、`XGBoost`
  
2. **Bagging（平行組合）**：
  - 分類器 classifier：  
    每個 classifier 可以並行生成，但每個模型要略有不同；  
    每個 classifier 權重一致，最後用投票的方式（Majority vote）取得最終結果  

    classifier_1 \   
    classifier_2 ──> Majority vote -> 結果  
    classifier_3 /  

  - 訓練樣本 train_df：  
    每次選擇的 train_df 都是隨機抽取，且每次每個訓練樣本的權重一致

  - 例如：`RandomForestClassifier` 隨機森林

## step1: 準備訓練資料

In [1]:
import pandas as pd

train_df = pd.read_csv("titanic/train.csv", encoding="utf-8")
test_df = pd.read_csv("titanic/test.csv", encoding="utf-8")

### 觀察特徵類型：
- 無用：PassengerId(刪)
- 答案：Survived
- 數值：Age, SibSp, Parch, Fare
- 類別：Pclass, Name(取出稱謂salutation), Sex, Ticket(刪，不知怎麼用), Cabin(刪，缺失值太多), Embarked

In [2]:
# 統計每個欄位缺失值（NA）數量：sum(False=0, True=1)
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
# 填補缺失值_訓練資料：數值型，填中位數
med = train_df.median()
train_df = train_df.fillna(med)

# 填補缺失值_測試資料：數值型，用訓練資料算出來的中位數
test_df = test_df.fillna(med)

In [4]:
# 填補缺失值_訓練資料：類別型，填出現最多的選項
most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)

# 填補缺失值_測試資料：類別型，用訓練資料找出來的的選項
test_df["Embarked"] = test_df["Embarked"].fillna(most)

In [5]:
def namecut(s):
    return s.split(",")[-1].split(".")[0].strip()

# 統計稱謂出現的的次數
salutation = train_df["Name"].apply(namecut).value_counts()
# 列出數量大於30的稱謂
reserved = salutation[salutation > 30].index
reserved

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [6]:
def namecut_2(s):
    s = s.split(",")[-1].split(".")[0].strip()
    if s in reserved:
        return s 
    else:
        return None

# namecut_2：稱謂數量大於30的保留，小於30的改為None
# reserved ：數量大於30的稱謂 = ['Mr', 'Miss', 'Mrs', 'Master']
train_df["Name"] = train_df["Name"].apply(namecut_2)
test_df["Name"] = test_df["Name"].apply(namecut_2)

In [7]:
# 對類別型特徵做 One-Hot Encoding
train_raw = pd.get_dummies(train_df, columns=["Name", "Sex", "Embarked"])
test_raw = pd.get_dummies(test_df, columns=["Name", "Sex", "Embarked"])

In [8]:
# 資料清洗 Data Cleansing
x_train = train_raw.drop(["PassengerId", "Survived", "Ticket", "Cabin"], axis=1)
y_train = train_raw["Survived"]
x_test = test_raw.drop(["PassengerId", "Ticket", "Cabin"], axis=1)
testid = test_raw["PassengerId"]

## step2: 建立訓練模型

### 交叉驗證（cross validation，CV）  
是用來選擇最佳或合適的 **預測參數**    

1. [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)  
  - 參數要自己提供（靠通靈XD），通常會先從 `max_depth=5, n_estimators=20` 開始慢慢試  
  - `cv=10` 表示把資料切割成 10 份，做 10 次交叉驗證，故會得到 10 個預測結果  
  - 再透過 `np.average(score)` 取得預測結果平均
  - 資料分割的差異：
    - `train_test_split()`：  
      是將 data 分成 2 份，一份做為 train_data，一份做為 test_data  
    - `cross_val_score()`：  
      是將 data 分成 X 份，每次選一份做為 test_data，剩下的 x-1 份做為 train_data  

2. [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)  
  - 暴力搜索版的交叉驗證  
  - 參數給一段 range，模型會把所有參數跑一遍，然後自己找出最佳解  
  - `cv=10` 表示要做 10 次交叉驗證，但只會給 1 個最佳預測結果

#### 交叉驗證 (第一種)：sklearn.model_selection.cross_val_score

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# 調整參數靠通靈 XD ────────────────────┐────────────────┐
clf = RandomForestClassifier(max_depth=8, n_estimators=26) 
# 交叉驗證（cross validation，CV）
# cv=10 表示要做 10 次交叉驗證
score = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)

print("score: \n", score)
print("average: ", np.average(score))

score: 
 [0.78888889 0.85393258 0.76404494 0.88764045 0.88764045 0.83146067
 0.82022472 0.7752809  0.88764045 0.84269663]
average:  0.8339450686641697


#### 交叉驗證 (第二種)：sklearn.model_selection.GridSearchCV

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()
params = {
    "max_depth":range(5, 11),
    "n_estimators":range(25, 40, 1)
}
# 交叉驗證（暴力搜索版）：參數給一段 range，讓模型自己找出最佳解
# cv=10 表示要做 10 次交叉驗證
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train, y_train)

print("best params: ", cv.best_params_)
print("best score: ", cv.best_score_)

best params:  {'max_depth': 8, 'n_estimators': 36}
best score:  0.8440449438202247


In [11]:
# 看起來最佳的參數
clf = RandomForestClassifier(max_depth=8, n_estimators=26)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=26,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## step3: 利用模型預測

In [12]:
pre = clf.predict(x_test)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "Survived":pre
})
# 不要儲存列編號：index=False
result_df.to_csv("titanic/predict_result_RF.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


### 觀察特徵重要性：`feature_importances_`

In [13]:
pd.DataFrame({
    "column":x_train.columns,
    "importance":clf.feature_importances_
}).sort_values("importance", ascending=False)

,column,importance
10,Sex_male,0.186441
4,Fare,0.161957
7,Name_Mr,0.132439
1,Age,0.124302
0,Pclass,0.098631
8,Name_Mrs,0.065875
2,SibSp,0.051795
9,Sex_female,0.049203
6,Name_Miss,0.037002
3,Parch,0.035602
